# Lesson 1: Building a ReAct Agent from Scratch

Welcome to this lesson on building a ReAct (Reasoning and Acting) agent from scratch. This content is adapted from the "AI Agents with LangGraph" course by [DeepLearning.AI](https://www.coursera.org/projects/ai-agents-in-langgraph), originally created by Harrison Chase (Co-Founder of [LangChain](https://www.langchain.com/)) and Rotem Weiss (Founder of [Tavily AI](https://tavily.com/)). I've modified the original course to use Amazon Bedrock and Anthropic's Claude model instead of OpenAI's GPT models.

This adaptation addresses a common challenge in AI development: the difficulty of porting applications between different Language Model (LLM) providers. Many developers find themselves locked into a single provider, often OpenAI, due to the complexities involved in transitioning to alternative models. Not only do we often need to adapt prompts to achieve maximum the best performance with each model, we should (or need) to adapt the output parsers depending on the model peculiarities. By walking through this conversion process, we aim to equip you with the skills to work with a diverse range of models, including open-source options.

If you are tasked with migrating to a new model, I highly encourage you to have the original notebook from the course open, to check for the differences.
When you are already an avid user of Amazon Bedrock, sit back and enjoy the learning experience!

Let's begin!

## The ReAct Pattern

In this lesson, we'll construct an AI agent using the ReAct (Reasoning and Acting) pattern. If you're unfamiliar with this concept, don't worry—we'll break it down step by step.

The ReAct pattern is a framework for structuring an AI's problem-solving process, mirroring human cognitive patterns:

1. **Reason** about the current situation
2. **Decide** on an action to take
3. **Observe** the results of that action
4. **Repeat** until the task is complete

To illustrate this concept, consider how an experienced software engineer might approach debugging a complex system:

1. **Reason**: Analyze the error logs and system state (e.g., "The database connection is timing out")
2. **Act**: Implement a diagnostic action (e.g., "Run a database connection test")
3. **Observe**: Examine the results of the diagnostic (e.g., "The test shows high latency")
4. **Repeat**: Continue this process, perhaps checking network configurations next, until the issue is resolved

Our AI agent will employ a similar methodology to tackle problems. As we develop this agent, pay attention to the division of labor between the AI model (the "brain" that reasons and decides) and our Python code (the "body" that interacts with the environment and manages the process flow).

This notebook is based on the following [notebook from Simon Willison](https://til.simonwillison.net/llms/python-react-pattern).


## Setting Up the Environment

Let's begin by importing the necessary libraries and configuring our environment.

### Initializing the Bedrock Client

To communicate with the Claude model via Amazon Bedrock, we need to establish a client connection. Think of this client as an API gateway that enables our code to send requests to the AI model and receive responses.

We'll use the boto3 library, which is the Amazon Web Services (AWS) SDK for Python. For those unfamiliar with AWS, boto3 can be thought of as a comprehensive toolkit that facilitates Python's interaction with various AWS services, including Bedrock.

<link>For comprehensive instructions on configuring boto3 with AWS credentials, refer to the AWS documentation</link>

In a production setting, you would implement secure AWS credential management. For the purposes of this lesson, we'll assume the credentials are pre-configured in your environment.


In [ ]:
import os
import sys
import re
import boto3
from botocore.config import Config
import warnings
warnings.filterwarnings("ignore")

# import local modules
dir_current = os.path.abspath('')
dir_parent = os.path.dirname(dir_current)
if dir_parent not in sys.path:
    sys.path.append(dir_parent)
from utils import utils

# Set basic configs
logger = utils.set_logger()
pp = utils.set_pretty_printer()

# Set main parameters

TAVILY_API_KEY_NAME = "TAVILY_API_KEY"
aws_region = os.getenv("AWS_REGION", default="us-east-1")

# Set bedrock configs
bedrock_config = Config(
    connect_timeout=120, read_timeout=120, retries={"max_attempts": 0}
)

# Create a bedrock runtime client in your aws region.
# If you do not have the AWS CLI profile setup, you can authenticate with aws access key, secret and session token.
# For more details check https://docs.aws.amazon.com/cli/v1/userguide/cli-authentication-short-term.html
bedrock_rt = boto3.client(
    "bedrock-runtime",
    region_name=aws_region,
    config=bedrock_config,
)

Lets set out some pararmeters


In [ ]:
# Set inference parameters
temperature = 0.5
top_k = 200
inference_config = {"temperature": temperature}
additional_model_fields = {"top_k": top_k}
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
system_prompts = [{"text": "You are a helpful agent."}]
message_1 = {"role": "user", "content": [{"text": "Hello world"}]}

# Instantiate messages list
messages = []
messages.append(message_1)

# Send the message.
response = bedrock_rt.converse(
    modelId=model_id,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config,
    additionalModelRequestFields=additional_model_fields,
)

pp.pprint(response)
print("\n\n")
pp.pprint(response["output"]["message"]["content"][0]["text"])

In [ ]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.system = [{"text": self.system}]
        self.bedrock_client = boto3.client(service_name="bedrock-runtime", region_name=aws_region)

    def __call__(self, message):
        self.messages.append({"role": "user", "content": [{"text": message}]})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": [{"text": result}]})
        return result

    def execute(self):
        inference_config = {"temperature": 0.5}
        additional_model_fields = {"top_k": 200}

        response = self.bedrock_client.converse(
            modelId="anthropic.claude-3-sonnet-20240229-v1:0",
            messages=self.messages,
            system=self.system,
            inferenceConfig=inference_config,
            additionalModelRequestFields=additional_model_fields,
        )
        return response["output"]["message"]["content"][0]["text"]

In [ ]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [ ]:
def calculate(what):
    return eval(what)


def average_dog_weight(name):
    if name in "Scottish Terrier":
        return "Scottish Terriers average 20 lbs"
    elif name in "Border Collie":
        return "a Border Collies average weight is 37 lbs"
    elif name in "Toy Poodle":
        return "a toy poodles average weight is 7 lbs"
    else:
        return "An average dog weights 50 lbs"


known_actions = {"calculate": calculate, "average_dog_weight": average_dog_weight}

In [ ]:
abot = Agent(prompt)

In [ ]:
result = abot("How much does a toy poodle weigh?")
print(result)

In [ ]:
result = average_dog_weight("Toy Poodle")
result

In [ ]:
next_prompt = "Observation: {}".format(result)

In [ ]:
abot(next_prompt)

In [ ]:
abot.messages

In [ ]:
abot = Agent(prompt)

In [ ]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
result = abot(question)
print(result)

In [ ]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

In [ ]:
result = abot(next_prompt)
print(result)

In [ ]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

In [ ]:
abot(next_prompt)

In [ ]:
result = abot(next_prompt)
print(result)

### Add loop


In [ ]:
action_re = re.compile(
    "^Action: (\w+): (.*)$"
)  # python regular expression to selection action

In [ ]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split("\n") if action_re.match(a)]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [ ]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

# Exercise - Rewrite the Agent to use Anthropic style prompting!


In [ ]:
# Hints to be added + solution for evaluation.